In [44]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

In [45]:
# Constants
NUM_PARTICLES = 20
SPACE_SIZE = 100
RADIATION_SOURCE = np.random.uniform(0, SPACE_SIZE, 2)
MAX_ITERATIONS = 20
THRESHOLD = 1.0
C = 1000  # Radiation constant
W = 0.5  # Inertia weight
C1 = 1.5  # Personal attraction coefficient
C2 = 1.5  # Global attraction coefficient

In [46]:
# Particle class
class Particle:
    def __init__(self):
        self.position = np.random.uniform(0, SPACE_SIZE, 2)
        self.velocity = np.random.uniform(-1, 1, 2)
        self.personal_best_position = np.copy(self.position)
        self.best_radiation = self.get_radiation()

    def get_radiation(self):
        distance = np.linalg.norm(self.position - RADIATION_SOURCE)
        if distance == 0:
            return float('inf')
        return C / (distance ** 2)

    def update_personal_best(self):
        current_radiation = self.get_radiation()
        if current_radiation > self.best_radiation:
            self.personal_best_position = np.copy(self.position)
            self.best_radiation = current_radiation

    def update_velocity(self, global_best_position):
        r1, r2 = np.random.rand(2)
        cognitive_component = C1 * r1 * (self.personal_best_position - self.position)
        social_component = C2 * r2 * (global_best_position - self.position)
        self.velocity = W * self.velocity + cognitive_component + social_component

    def update_position(self):
        self.position += self.velocity
        self.position = np.clip(self.position, 0, SPACE_SIZE)

In [53]:
# Initialize particles
particles = [Particle() for _ in range(NUM_PARTICLES)]

In [54]:
# PSO implementation
def particle_swarm_optimization():
    global_best_position = max(particles, key=lambda p: p.best_radiation).personal_best_position
    global_best_radiation = max(particles, key=lambda p: p.best_radiation).best_radiation

    fig, ax = plt.subplots()
    scatter_particles = ax.scatter([], [], color='blue', label='Particles')
    scatter_gbest = ax.scatter([], [], color='green', marker='o', label='Global Best')
    scatter_source = ax.scatter(*RADIATION_SOURCE, color='red', marker='x', label='Radiation Source')

    def init():
        ax.set_xlim(0, SPACE_SIZE)
        ax.set_ylim(0, SPACE_SIZE)
        ax.legend()
        return scatter_particles, scatter_gbest, scatter_source

    def update(frame):
        nonlocal global_best_position, global_best_radiation

        previous_global_best_radiation = global_best_radiation

        for particle in particles:
            particle.update_velocity(global_best_position)
            particle.update_position()
            particle.update_personal_best()

        # Update global best
        current_global_best_particle = max(particles, key=lambda p: p.best_radiation)
        if current_global_best_particle.best_radiation > global_best_radiation:
            global_best_position = current_global_best_particle.personal_best_position
            global_best_radiation = current_global_best_particle.best_radiation

        # Update scatter plot data
        positions = np.array([particle.position for particle in particles])
        scatter_particles.set_offsets(positions)
        scatter_gbest.set_offsets(global_best_position)

        ax.set_title(f'Iteration {frame + 1}')

        # Check termination condition
        if np.linalg.norm(global_best_position - RADIATION_SOURCE) < THRESHOLD:
            ani.event_source.stop()
            return scatter_particles, scatter_gbest

        # Check if all particles are within the threshold distance to the source
        if all(np.linalg.norm(particle.position - RADIATION_SOURCE) < THRESHOLD for particle in particles):
            ani.event_source.stop()

        # Check if global best radiation has stopped improving
        if global_best_radiation == previous_global_best_radiation:
            ani.event_source.stop()

        return scatter_particles, scatter_gbest

    ani = animation.FuncAnimation(fig, update, frames=MAX_ITERATIONS, init_func=init, blit=True, repeat=False)
    plt.close(fig)
    return ani

In [55]:
# Run PSO and display animation
ani = particle_swarm_optimization()
ani.save('particle_swarm_optimization.gif', writer='pillow')
# HTML(ani.to_jshtml())